In [1]:
from envs.test_env import Electric_Car
from models.baseline import BaselineModel
from models.baseline2 import BaselineModel2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [4]:
def learn(env, model):
    states, rewards, infos = [], [], []
    truncated = False
    terminated = False
    while(terminated == False and truncated == False):
        action = model.predict(env)
        obs, reward, termination, truncation, info = env.step(action)
        states.append(obs)
        rewards.append(reward)
        terminated = termination
        truncated = truncation
        infos.append(info)

    
    return states, rewards, terminated, truncated, infos